# Web Scrapping to CoE

## Import Libraries

In [1]:
import sys
sys.path.insert(1, '../shared')
import requests
from bs4 import BeautifulSoup
from time import sleep

import pandas as pd

import numpy as np

import os.path
from os import path

from utils import verbosity

from typing_extensions import List, Dict

import pandas as pd
df = pd.DataFrame()

import math



## Define WebScrapping atributes

In [ ]:
verbosity(msg=f'getting_link',
            tl=1)

auction_link = 'https://allianceforcoffeeexcellence.org/competition-auction-results/'

is_verbosity = True

## Define other classess

### Attributes dictionary

In [3]:
class AttributesDictionary:
    
    
    def __init__(self):
        self.dict_auction_translator = {
            'Country': [''],
            'Rank': ['RANK', 'Ranking', 'LOT NUMBER', 'Rank', 'LOT #', 'Lot #', 'Lot Number', 'LOT NO.', 'Lot number'],
            'Score': ['SCORE', 'Score'],
            'Farm':['FARM', 'Farm', 'Farm / CWS', 'Winning Farm / CWS', 'FARM NAME', 'Farm Name', 'FARM / CWS', 'Nome', 
                    'FARM/CWS', 'WINNING FARM / CWS', 'FARM / CWS'],
            'Url_farm':[''],
            'Farmer':['FARMER', 'Farmer / Representative', 'PRODUCER', 'Farmer', 'FARMER / REPRESENTATIVE', 
                      'Producer', 'Title Farmer / Organization', 'Farmer Name'],
            'Region':['REGION', 'Region'],
            'Weigth_(Kg)':['WEIGHT (KG)', 'Weight (kg)', 'Weight (KG)', 'WEIGHT (kg)', 'Weight', 'WEIGHT (Kg)', 
                           'Weight (Kg)', 'WEIGHTS (Kg)', 'Estimated Weight (kg)', 'WEIGTH (Kg)', 'Auction Lot Size (kg)',
                           ],
            'Weigth_(Lb)':['Weight – lbs', 'Pounds', 'Weight – Lbs', 'Weight (lbs)', 'Weight (lbs.)', 'Weight (Lbs)', 
                           'WEIGHT (lb)', 'POUNDS (lb)', 'Weight(Lb)', 'Weight (lb)', 'Weight(Lbs)'],
            'Variety':['VARIETY', 'Variety', 'Variedad', 'Variety', 'Variety '],
            'Processing':['PROCESS', 'Process', 'Processing', 'Process, Variety', 'PROCESSING', ' Variety/Processing',
                          'Proceso', 'Processing system', 'Processing System'],
            'High_Bid':['High Bid', 'HIGH BID', ' Price per lb', 'Price per lb', '\xa0Price per lb', '&nbsp;Price per lb', 
                        'Bid ($/lb)', 'Bid', ' FINAL BID ($/lb)', 'HIGHEST BID', 'Hight Bid', 'High Bid ($/lb)', 
                        'Total value', 'PRICE PER (lb)', 'HighBid', 'High bid'],
            'High_Bidders':['High Bidder(s)', 'HIGH BIDDER(S)', 'Winner', 'High bidder(s)', 'High bidder', 'High bidders' ],
            'Origin_Bidder':['Origin', 'Buyer’s Location', 'COUNTRY', 'Country', 'Country of Origin'],
            'Total_Value':['Total Value', 'TOTAL VALUE', 'Total Price', ' Total Value', 'Total Value ($)', 'TOTAL PRICE',
                           ' Total Price', ' Total Value'],
            'Total_Comission':['TOTAL COMISSION', 'Commissions', 'Auction Commission', 'Commission', 'Comissions', 
                               'COMMISSION', ' COMISSION', 'Total Commissions', 'AUCTION COMMISSION', ' Commission'],
            'Company_Name':['Company Name'],
            'Size_30Kg_boxes':['Size', 'Lot Size', 'Size (30kg Boxes)', 'SIZE', 'Boxes', 'LOT SIZE', 
                               'Size (30kg boxes)'],
            'Altitude':['ALTITUDE (masl)', 'ALTITUDE (m)', 'Altitude'],
            'History':[''],
            'Enviromental_Care':[''],
            'Coffee_Processing_Information':[''],
            'Annual_Production':[''],
            'Annual_Precipitation':[''],
            'Shade_Grown_Type':[''],
            'Water_Source':[''],
            'Coffee_Growing_Area':['Coffee Growing Area ', 'Coffee Growing Area'],
            'Aroma_Flavor': ['Aroma/Flavor', 'Aroma / Flavor'],
            'Acidity': ['Acidity'],
            'Overall': ['Overall', 'Coffee Characteristics'],
            'Other': ['Other'],
            'Farm_Size': ['Farm Size'],
            'Auction_Lot_Size': ['Auction Lot Size (lbs.)'],
            'Auction_Lot_Size(Kg)': ['Auction Lot Size (kg)'],
            'Certifications': ['Certifications'],
            'City': ['City'],

        }

    def translate_attribute(self, value_attribute):
        name_attribute = ''

        for item in self.dict_auction_translator.items():
            if value_attribute in item[1]:
            # for attribute in item[1]:
            #     if attribute == value_attribute:
                name_attribute = item[0]
                
                return name_attribute
        if name_attribute=='':
            verbosity(f"Title {value_attribute} not found; it will be added:", tl=5)            
            self.dict_auction_translator[value_attribute]=[value_attribute]
            # return value_attribute
                    

### Other classes

In [4]:
def find_number_in_string_array(array):
    for value in array:
        if value.isdigit():
            return value

class CharProcessing:
    
    @staticmethod
    def get_country_name(auction_link):
        auction_link=auction_link[:(len(auction_link)-1)]
        array_auction_split = auction_link.split('/')
        return array_auction_split[(len(array_auction_split))-1]
        
class SearchOperations:
    
    @staticmethod
    def get_country_by_name(array_countries, name):
        '''Returned argument is the country and link auction'''
        for country in array_countries:
            if(country['name']==name):
                return country


class SaveOperations:
    def __init__(self):
        pass
    
    
    def file_exist(self, file_name):
        
        return path.exists(f'../static/{file_name}')
    
    def save_html_page(self, html_code, file_name):
        verbosity(msg=f'enter in save_html_page function',
                    tl=1)
        verbosity(msg=f'checking if the file exist',
                    tl=2)
        if(self.file_exist(file_name)):
            verbosity(msg=f'the file exist',
                        tl=2)
            verbosity(msg=f'saving html page',
                        tl= 3)
            f = open(f'../static/{file_name}','w')
            f.write(html_code)
            f.close()
        else:
            verbosity(msg=f"the file doesn't exist",
                        tl=2)
            verbosity(msg=f"creating file",
                        tl=3)
            f = open(f'../static/{file_name}', "x")
            verbosity(msg=f"saving html page",
                        tl=3)
            f.write(html_code)
            f.close()
        verbosity(msg=f'out from save_html_page function',
                    tl=1)

class Page:
    
    
    def __init__(self, root_link:str):
        verbosity(msg=f'Page class was initialized with root_link: {root_link}',
                    tl=1)
        self.root_link = root_link
        is_url = not(self.is_nan(root_link))
        if is_url:
            self.web_content = self.get_web_content()
            self.found_404 = self.find_404()
            self.attributes_dictionary = AttributesDictionary()
        else:
            self.found_404 = True
            self.web_content=False

    def is_nan(self, value):
        if(type(value)==float):
            if math.isnan(value):
                return True
            
        return False
        
    def aux_found_404(self, value):
        verbosity(value)
        if value == '404':
            raise ValueError('Error 404 in web page')


    def find_404(self, tl_ini=5):
        web_b_list = self.get_content_by_html_tags(self.web_content, 'b')
        try:
            [self.aux_found_404(web_b.text) for web_b in web_b_list]
        except ValueError as e:
            verbosity(f"Value Error: {e}", level='error', tl=tl_ini)
            return True
        return False

        
    def get_web_content(self, tl=2):
        ''' returned the web content of a link with Beautiful Soup '''
        verbosity(msg=f'getting web content', 
                    verb=is_verbosity, tl=tl)
        sleep(0.5)
        verbosity(msg=f'request page from link',
                    verb=is_verbosity, tl= tl+1)
        try:
            response = requests.get(self.root_link)
            verbosity(msg=f'Success request',
                        verb=is_verbosity, tl= tl+2)
        except Exception as e:
            verbosity(msg = f'error: {e}',
                    level='error',	verb=is_verbosity, tl= tl+1)
            return False
            
        verbosity(msg=f'returning content',
                    verb=is_verbosity, tl= tl+2)
        
        web_content = BeautifulSoup(response.content, 'html.parser')
        
        return web_content
    
    def get_content_by_html_tags(self, beuty_object, html_tag):
        ''' get the web content of beautiful object by html tag '''
        return beuty_object.find_all(html_tag)
    
    
    def get_content_by_class(self,  beuty_object, class_name,):
        ''' get the web content of beautiful object by class '''
        return beuty_object.find_all(class_=class_name)
    
       
    def get_content_by_html_attr(self,  beuty_object, attribute_name, attribute_value):
        ''' get the web content of beautiful object by attribute '''
        return beuty_object.find_all(attrs={attribute_name : attribute_value})
    
    def get_p_text_from_id(self, id_value, tl_ini = 5):
        try:
            value = self.get_content_by_html_attr(self.web_content, 'id', id_value)[0]
            value = self.get_content_by_html_tags(value, 'p')[0].text
        except IndexError as e:
            verbosity(f"Value not found with id: {id_value}", level='error', tl=tl_ini)
            value = ''

        return value
    
    def concat_elements_from_result(self, list):
        text = ""
        for value in list:
            text = text + "\n" + value.text
        return text


        



## Define WebScrapping classess

### Class to Home Page

In [5]:
class MainPage (Page):
    
        
    def get_auction_links(self, web_content_main, id_main_menu):
        '''
        Summary: 
            get the links of auctions in main page
        
        Parameters:
            web_content_main: content of main page
            id_main_menu: id to search all links in main menu
            
        Returns:
            list with the auctions links
        
        '''
        links = web_content_main.find_all(id=id_main_menu)[0].find_all('a')
        array_auctions_links = []
        
        index = 0
        for link in links[:(len(links)-1)]:
            link_href = str(link.get('href'))
            if(link_href!='#' and link_href != 'None'):
                array_auctions_links.append(
                    {
                        'name': CharProcessing.get_country_name(link_href),
                        'link': link_href
                    }
                )
                                
                
                # array_auctions_links['name'].append(CharProcessing.get_country_name(link_href))
                # array_auctions_links['link'].append(link_href)
        return array_auctions_links
                
                # index += 1
            
        



### Auction Data Class

In [6]:

class Auction:
    def __init__(self):
        self.dict_auction = {
            'Year': '',
            'Country': '',
            'Rank': '',
            'Score': '',
            'Farm':'',
            'Url_farm':'',
            'Farmer':'',
            'Region':'',
            'Weigth_(Kg)':'',
            'Weigth_(Lb)':'',
            'Variety':'',
            'Processing':'',
            'High_Bid':'',
            'High_Bidders':'',
            'Origin_Bidder':'',
            'Total_Value':'',
            'Total_Comission':'',
            'Company_Name':'',
            'Size_30Kg_boxes':'',
            'Altitude':'',
            'History':'',
            'Enviromental_Care':'',
            'Coffee_Processing_Information':'',
            'Annual_Production':'',
            'Annual_Precipitation':'',
            'Shade_Grown_Type':'',
            'Water_Source':'',
            'Coffee_Growing_Area':'',
            'Aroma_Flavor':'',
            'Acidity':'',
            'Overall':'',
            'Other':'',
            'Farm_Size':'',
            'Auction_Lot_Size':'',
            'Auction_Lot_Size(Kg)':'',
            'Certifications':'',
            'City':'',

        }

        
    def get_dict_auction(self):
        return self.dict_auction
    
    def set_dict_auction(self, dict_auction):
        self.dict_auction = dict_auction
    
        
    
    def __str__(self):
        string = ("Country: " +self.dict_auction['Country'] + 
                    "\n" + "Rank: " +self.dict_auction['Rank'] + 
                    "\n" + "Score: " + self.dict_auction['Score'] + 
                    "\n" + "Farm: " + self.dict_auction['Farm'] + 
                    "\n" + "Url_farm: " + self.dict_auction['Url_farm'] + 
                    "\n" + "Farmer: " + self.dict_auction['Farmer'] + 
                    "\n" + "Region: " + self.dict_auction['Region'] + 
                    "\n" + "Weigth_(Kg): " + self.dict_auction['Weigth_(Kg)'] + 
                    "\n" + "Weigth_(Lb): " + self.dict_auction['Weigth_(Lb)'] + 
                    "\n" + "Variety: " + self.dict_auction['Variety'] + 
                    "\n" + "Process: " + self.dict_auction['Process'] + 
                    "\n" + "High_Bid: " + self.dict_auction['High_Bid'] + 
                    "\n" + "Total_Value: " + self.dict_auction['Total_Value'] + 
                    "\n" + "Company_Name: " + self.dict_auction['Company_Name'] + 
                    "\n" + "Total_Comission: " + self.dict_auction['Total_Comission'] + 
                    "\n")
        return string
                
        
class AuctionData:
    
    def __init__(self):
        self.array_auctions = []
        self.auctions_number = 0
        self.tl = 3
    
    
    def set_auctions_number(self, auctions_number):
        self.auctions_number = auctions_number
    
    
    def get_auctions_number(self):
        return self.auctions_number 
    
    def compare_auction_index(self, auction, rank):
        if auction.get_dict_auction()['Rank'].lower() == rank.lower():
            return True
        else:
            return False

    
    def get_dict_auction_index_by_rank(self, rank):
        try:
            dict_auction_index = [self.compare_auction_index(auction, rank) for auction in self.array_auctions].index(True)
            return dict_auction_index
        
        except ValueError as e:
            # verbosity(f'Value Error in get_dict_auction_index_by_rank function: {e} in rank {rank} ', level='error', tl=self.tl)
            return 'False'
        
        except AttributeError as e:
            verbosity(f'Attribute Error in get_dict_auction_index_by_rank function: {e}', level='error', tl=3)
            return 'False'
    
    
    
    def refactor_array_auctions(self):
        # print(self.array_auctions)
        array_auctions_complete = []
        try:
            array_auctions_complete = [auction.get_dict_auction() for auction in self.array_auctions]
        except AttributeError:
            verbosity(msg=f'error detected in Auction object', level='Error', tl=1)
        
        return(array_auctions_complete)
        # print(self.array_auctions_complete)
    
    def get_data_titles(self):
        dict_auction = self.array_auctions[0].get_dict_auction()
        verbosity(msg=f'getting title_auctions',
                    tl=1)
        array_titles = [title_auction for title_auction in dict_auction.keys()]
        verbosity(msg=f'getting title_auctions - Success!',
                    tl=1)
        return array_titles

        

### Class to Auction Results Page

In [7]:

class AuctionResultsPage(Page):
    
    def __init__(self, dict_page_country:Dict):
        self.root_link = dict_page_country['link']
        super().__init__(self.root_link)
        array_name_page = dict_page_country['name'].split('-')
        self.country = dict_page_country['name']
        self.year = find_number_in_string_array(array_name_page)
        verbosity(msg=f'Page class was initialized with. \n root_link: {self.root_link} \n country: {self.country} \n year: {self.year}',
                    verb=is_verbosity, tl=1)
    
    def get_table_titles (self, array_td, attributes_dictionary):
        array_titles = []
        for td in array_td:
            array_titles.append(attributes_dictionary.translate_attribute(td.text))
        return array_titles
    
    def get_auction_data(self, web_content_main, tl=3):
        if self.web_content==False:
            pass
            
        verbosity(msg=f'getting auction data',
            verb=is_verbosity, tl=tl)
        auction_data = AuctionData()
        attributes_dictionary = AttributesDictionary()
        array_table = self.get_content_by_html_tags(web_content_main, 'table')

        for table_data in array_table:
            try:
                array_table_body = self.get_content_by_html_tags(table_data, 'tbody')[0]
            except IndexError:
                continue
            array_table_head = self.get_content_by_html_tags(table_data, 'thead')
            array_tr = self.get_content_by_html_tags(array_table_body, 'tr')
            title_in_initial_tr = False
            array_titles = []
            

            if len(array_table_head)!=0:
                    array_th = self.get_content_by_html_tags(array_table_head[0], 'th')
                    array_titles = self.get_table_titles(array_th, attributes_dictionary)
            else:
                array_td = self.get_content_by_html_tags(array_tr[0], 'td')
                array_titles = self.get_table_titles(array_td, attributes_dictionary)
                title_in_initial_tr = True

            for tr in array_tr:
                array_td = self.get_content_by_html_tags(tr, 'td')
                is_fake = False
                auction_index = 'False'
                auction = Auction()
                if title_in_initial_tr:
                    title_in_initial_tr = False
                
                else:
                    for td, table_title in zip(array_td, array_titles):
                        if table_title == 'Farm':
                            a_tag = td.find('a', href=True) #self.get_content_by_html_tags(td, 'a')[0]
                            if(a_tag != None):
                                auction.dict_auction['Url_farm'] = a_tag['href']
                            auction.dict_auction[table_title] = td.text
                        else:
                            if table_title == 'Rank':
                                auction_index = auction_data.get_dict_auction_index_by_rank(td.text)    
                                
                                if (str(td.text)==''):
                                    is_fake = True
                                auction.dict_auction[table_title] = (td.text.lower())
                            
                            else:
                                auction.dict_auction[table_title] = td.text
                            
                            
                    if(not(is_fake)):
                        if(auction_index!='False'):
                            for key, value in auction.get_dict_auction().items():
                                if(value!=''):
                                    auction_data.array_auctions[auction_index].get_dict_auction()[key] = value
                                                                
                            # auction.set_dict_auction(auction_data.array_auctions[auction_index].get_dict_auction() | auction.get_dict_auction())
                        else:
                            auction.dict_auction['Country'] = self.country
                            auction.dict_auction['Year'] = self.year
                            try:
                                auction.get_dict_auction()['Rank'][0].isnumeric()
                                auction_data.array_auctions.append(auction)
                            except:
                                pass
                        
        return auction_data                      
            
            

## Main Program to extract auctions data

In [8]:
def get_data_of_countries(country_to_find_data=''):
    '''
    Summary:
        get all data of country auction tables

    Parameters: 
        country_to_find_data: name of the country and year ie. brazil-2023, brazil

    '''
    country_to_find_data = '' # if empty, then search all ie. 'brazil-2023'
    auction_file_name = 'Auctions.xlsx'
    ## Web Scrapping
    auction = Auction()
    mainPage = MainPage(auction_link)
    web_content_main = mainPage.get_web_content()   
    array_auctions_links = mainPage.get_auction_links(web_content_main, 'menu-coe-country-programs-menu')
    df_auctions = pd.DataFrame(columns=auction.dict_auction.keys())
    if country_to_find_data:
        array_auctions_links = [auction for auction in array_auctions_links if country_to_find_data in auction['name'] ]
        auction_file_name = f"Auctions-{country_to_find_data}.xlsx"

    else: 
        array_auctions_links = [auction for auction in array_auctions_links ]
    for country in array_auctions_links:
        auction_page = AuctionResultsPage(country)
        web_content_auction_page = auction_page.get_web_content()
        auction_data = auction_page.get_auction_data(web_content_auction_page)
        array_dict_auctions = auction_data.refactor_array_auctions()
        df = pd.DataFrame(array_dict_auctions)
        df_auctions = pd.concat([df_auctions, df], ignore_index=True)
        # verbosity(msg=f"df_auctions = \n {df_auctions.tail()}", tl=1)

    df_auctions.to_excel(f"../auctions_download/{auction_file_name}")

# get_data_of_countries()


## Main Program to extract deep auctions data

### Deep Auction Page Class

In [9]:
class DeepAuctionPage(Page):

    def __init__(self, df_auction_row):
        root_link = df_auction_row['Url_farm']
        super().__init__(root_link)
        self.auction_data = {
            'History':'',
            'Enviromental_Care':'',
            'Coffee_Processing_Information':'',
            'Annual_Production':'',
            'Annual_Precipitation':'',
            'Shade_Grown_Type':'',
            'Water_Source':'',
            'Coffee_Growing_Area':'',
            'Aroma_Flavor':'',
            'Acidity':'',
            'Overall':'',
            'Other':'',
            'Farm_Size':'',
            'Auction_Lot_Size':'',
            'Auction_Lot_Size(Kg)':'',
            'Certifications':'',
            'City':'',
        }
        self.df_auction_row = df_auction_row



    def save_value(self, key_dict, value, tl_ini=3):

        try:
            if self.is_nan(self.df_auction_row[key_dict]):
                if value!='':
                    self.auction_data[key_dict] = value
                
        except KeyError as e:
            verbosity(f"Key dict:-{key_dict}-not found", tl=tl_ini)

    def get_description_from_auction(self):
        description_value = self.get_p_text_from_id('listing-description')
        data_table = self.get_content_by_html_attr(self.web_content, 'id', 'listing-details')

        if(description_value=='' and len(data_table)==0):
            web_description_div=self.get_content_by_class(self.web_content, 'block-field-job_description')
            if len(web_description_div)>=1:
                web_description_div=web_description_div[0]
                web_description_body=self.get_content_by_class(web_description_div, 'pf-body')[0]
                web_description_paragraph_list=self.get_content_by_html_tags(web_description_body, 'p')
                description_value = self.concat_elements_from_result(web_description_paragraph_list)
        self.save_value('History', description_value)

    def get_data_from_horizontal_table(self):
        data_table = self.get_content_by_html_attr(self.web_content, 'id', 'listing-details')[0]
        data_table_title = self.get_content_by_html_tags(data_table, 'th')
        data_table_value = self.get_content_by_html_tags(data_table, 'td')
        for title, value in zip (data_table_title, data_table_value):
            key_dict = self.attributes_dictionary.translate_attribute(title.text)
            if(key_dict):
                self.save_value(key_dict, value.text)

    def get_data_from_horizontal_list(self):
        web_data_list = self.get_content_by_html_tags(self.web_content, 'li')
        for web_li in web_data_list:
            web_data_list_div = self.get_content_by_html_tags(web_li, 'div')
            if len(web_data_list_div)==2:
                key_dict = self.attributes_dictionary.translate_attribute(web_data_list_div[0].text)
                if(key_dict):
                    self.save_value(key_dict, web_data_list_div[1].text)

    def aux_add_2_dataframe(self, auction_key, auction_value, tl_ini=6):
        self.df_auction_row[auction_key] = auction_value

    def add_2_dataframe(self, tl_ini=5):
        dict_data = self.auction_data
        [self.aux_add_2_dataframe(auction_key, auction_value) 
            for auction_key, auction_value 
            in zip(dict_data.keys(), dict_data.values()) ]
    
    def get_auction_data(self, tl_ini=2): 
        self.get_description_from_auction()
        try:
            self.get_data_from_horizontal_table()
        except IndexError:
            self.get_data_from_horizontal_list()
        self.add_2_dataframe()

    



### Read table

### get all deep data

In [ ]:
min_row_index = 4001
max_row_index = 6000

df_auctions_table = pd.read_excel(r'../auctions_download/AuctionsV2.xlsx', index_col=0)

def get_all_deep_data():
    for row_index, df_auction_row in df_auctions_table.iterrows():
        if(min_row_index <= row_index and row_index <= max_row_index):
            verbosity(f"Loading data from: {df_auction_row['Country']} {df_auction_row['Year']}", level='notif', tl=1)
            verbosity(f"Index: {row_index}", level='notif', tl=1)
            deep_auction_page = DeepAuctionPage(df_auction_row)
            if(deep_auction_page.web_content!=False and not(deep_auction_page.found_404)):
                deep_auction_page.get_auction_data(df_auction_row)
                df_auctions_table.iloc[row_index] = df_auction_row

get_all_deep_data()
df_auctions_table.to_excel("../auctions_download/AuctionsV2.xlsx", engine='xlsxwriter')


In [11]:
# df_auctions_table.to_excel("../auctions_download/AuctionsV2.xlsx", engine='xlsxwriter')


In [12]:
# 


In [13]:
# save_operations = SaveOperations()
# save_operations.save_html_page(html_code=str(web_content_auction_page), file_name='html_code_auction_page.html')
